In [ ]:
import pandas as pd
import os 
import numpy as np

In [ ]:
# Location of training file for our training dataset
# train_file = "../input/landmark-recognition-2021/train.csv"
# Reading file using pandas "read_csv" 
# data = pd.read_csv(train_file)
# Exploring the dataset
# data.head()

In [ ]:
# print(data["landmark_id"].max(), data["landmark_id"].min(), data["landmark_id"].nunique(), data["landmark_id"].unique())

In [ ]:
# seems like our data containes label inconsistent so we need a mapping dictiory for mapping our target values associated with 
#our input_image

# reverse_target_mapping = { i : value for i,value in enumerate(sorted(data["landmark_id"].unique()))}

In [ ]:
# reverse_target_mapping

In [ ]:
# Here we are exchaning the keys and values in each other for mapping purpose
# target_mapping = {value : key for key,value in zip(reverse_target_mapping.keys(), reverse_target_mapping.values())}

In [ ]:
# Sorting the value for concatnating our input image train file for processing
# data.sort_values(inplace = True, by = "id")
# Reseting the indices
# data.reset_index(inplace = True)
# droping the previous indices column , this can be done using reset index = True, but i forgot
# data.drop(columns = ["index"] , inplace = True)

# data.head()

In [ ]:
# data["landmark_id"] = data["landmark_id"].map(target_mapping)
# data.head()

In [ ]:
# filepath = "../input/landmark-recognition-2021/train/"
# images = list()
# with os.scandir(filepath) as entries_1:
#     for entry_1 in entries_1:
#         if not entry_1.is_file():
#             with os.scandir(filepath + entry_1.name) as entries_2:
#                 for entry_2 in entries_2:
#                     if not entry_2.is_file():
#                         with os.scandir(filepath + entry_1.name + '/' + entry_2.name) as entries_3:
#                             for entry_3 in entries_3:
#                                 if not entry_3.is_file():
#                                     with os.scandir(filepath + entry_1.name + '/' + entry_2.name + '/' + entry_3.name) as entries_4:
#                                         for entry_4 in entries_4:
#                                             images.append(filepath + entry_1.name + '/' + entry_2.name + '/' + entry_3.name + '/' + entry_4.name)
# print(len(images))
#                 for image in images:
#                     fileimages.append(filepath + entry.name + '/' + entry.name + '/' + image.name)

In [ ]:
# series = pd.DataFrame(data = {"true_label" : target_mapping.keys(), "mapped_label" : target_mapping.values()})
# print(series)
# series.to_csv("./label_mapping.csv")


# data["input_file_loc"] = sorted(images)
# data.head()
# data.to_csv('./train_target.csv', index=False)
data = pd.read_csv('../input/train-target/train_target.csv',usecols=["landmark_id", "input_file_loc"])
# data = data.drop(index=range(1,1000000), inplace=False)
data.head()

In [ ]:
data.iloc[0, 1]

In [ ]:
import sklearn.model_selection
import torchvision.datasets
import torchvision.transforms as transforms
import torch
import torchvision
from PIL import Image

transform = transforms.Compose(
    [transforms.Resize((28, 28)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
filepath = "../input/landmark-recognition-2021/train/"

# Create the Torch Dataset object for our pandas dataframe
class GLRDataset(torch.utils.data.Dataset):
    def __init__(self, data, root_dir, transform):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if self.root_dir == '../input/landmark-recognition-2021/train/':
            img_name = self.data.iloc[idx, 1]
            y_label = torch.tensor([self.data.iloc[idx, 0]])
        elif self.root_dir == '../input/landmark-recognition-2021/test/':
            img_name = self.data.iloc[idx, 0]
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)
        if self.root_dir == '../input/landmark-recognition-2021/train/':
            return (image, y_label)
        elif self.root_dir == '../input/landmark-recognition-2021/test/':
            return image

In [ ]:
dataset = GLRDataset(data=data, root_dir=filepath, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=100, shuffle=True, num_workers=0)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.shape, labels.shape)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import torchvision.models as models

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# model = EfficientNet.from_pretrained('efficientnet-b0')
# model.to(device)

import math
import torch.utils.model_zoo as model_zoo


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out



class CNN(nn.Module):

    def __init__(self, block, layers, num_classes=81313):
        self.inplanes = 64
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))

        self.convtranspose1 = nn.ConvTranspose2d(512, 2048, kernel_size=3, stride=1, padding=1, output_padding=0,
                                                 groups=1, bias=False, dilation=1)

        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        self.fclass = nn.Linear(2048, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        # 新加层的forward
        x = self.convtranspose1(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fclass(x)
        return x


    
    
resnet34 = models.resnet34(pretrained=False)
resnet34.load_state_dict(torch.load('../input/resnet34/resnet34-333f7ec4.pth'))

net = CNN(BasicBlock, [3, 4, 6, 3])
net = net.to(device)

pretrained_dict = resnet34.state_dict()
model_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

model_dict.update(pretrained_dict)

net.load_state_dict(model_dict)

# class GLRModel(nn.Module):
#     def __init__(self):
#         super(GLRModel, self).__init__()
#         self.layer1 = nn.Sequential(
#             # nn.BatchNorm2d(1280),
#             nn.Conv2d(3, 20, 3),        #(100, 20, 222, 222)
#             nn.BatchNorm2d(20),
#             nn.Conv2d(20, 20, 3),       #(100, 20, 220, 220)
#             nn.BatchNorm2d(20),
#             nn.ReLU(),
#             nn.AvgPool2d(2, 2))         #(100, 20, 110, 110)
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(20, 5, 3),        #(100, 5, 108, 108)
#             nn.BatchNorm2d(5),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2))         #(100, 5, 54, 54)
#         self.pool = nn.Conv2d(5, 16, 1) #(100, 16, 54, 54)
#         self.fc1 = nn.Linear(16 * 54 * 54, 81313)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.pool(x)
#         x = x.reshape(100, -1)
#         x = self.fc1(x)
#         x = F.relu(x)
#         # x = F.softmax(x, dim=1)
#         return x

    # net = CNN(BasicBlock, [3, 4, 6, 3])
# net = net.to(device)
# net = GLRModel()
# net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.005)

# optimizer = optim.RAdam(lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)

In [ ]:
from torch.autograd import Variable


# for epoch in range(1):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         # get the inputs
#         inputs, labels = data
#         # wrap them in Variable
#         inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))
#         labels = labels.flatten()
# #         print(labels.shape)
# #         labels = Variable(inputs.view(20,))

#         # forward + backward + optimize
#         outputs = net(inputs)

#         loss = criterion(outputs, labels)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
        
#         if i % 200 == 199:    # print every 20 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 200))
#             running_loss = 0.0
# torch.save(net.state_dict(), './model_GLR_dict')

In [ ]:
net.eval()
net.load_state_dict(torch.load("../input/model-glr-dict/model_GLR_dict"))

In [ ]:
# get all images paths
test_images = []
images_id = []
for dirpath, dirnames, filenames in os.walk("../input/landmark-recognition-2021/test"):
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        test_images.append(os.path.join(dirpath, filename))
        filename = os.path.splitext(filename)[0]
        images_id.append(filename)


test_df = pd.DataFrame(test_images, columns=['input_file_loc']) 
test_df.head()

In [ ]:
images_id = sorted(images_id)
test_df["input_file_loc"] = sorted(test_images)

In [ ]:
images_id[0:5]
test_df.head()

In [ ]:
test = GLRDataset(data=test_df, root_dir='../input/landmark-recognition-2021/test/', transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=100, shuffle=False, num_workers=0)

In [ ]:
dataiter = iter(test_loader)
images = dataiter.next()
print(images.shape, test.__len__())

In [ ]:
predict_conf_list = list()
predict_id_list = list()

for i, images in enumerate(test_loader):
    images = images.to(device)
    images = Variable(images)
    outputs = net(images)
    predict_id = torch.max(outputs.data, 1)[1].to(device).data.cpu().numpy()
    predict_conf = F.softmax(outputs, dim=1)
    predict_conf = torch.max(predict_conf.data, 1)[0].to(device).data.cpu().numpy()

    predict_conf = np.array(predict_conf) 
    predict_conf_list.append(predict_conf)
    predict_id = np.array(predict_id) 
    predict_id_list.append(predict_id)


In [ ]:
# predict_id_list = np.reshape(predict_id_list, (1,))
results = list()

pre_id_list = list()
for id in predict_id_list:
    for pre_id in id:
            pre_id_list.append(pre_id)

pre_conf_list = list()
for id in predict_conf_list:
    for pre_conf in id:
            pre_conf_list.append(pre_conf)


print(len(pre_conf_list))

for i in range(10345):
    results.append(str(pre_id_list[i]) + ' ' + str(pre_conf_list[i]))
images_id = pd.Series(images_id, name='id').astype(str)
images_Label = pd.Series(results, name='landmarks').astype(str)

test_df = pd.concat([images_id, images_Label], axis=1)

# data["predict_id"] = data["predict_id"].map(reverse_target_mapping)
test_df

In [ ]:
test_df.to_csv('submission.csv', index=False)

In [ ]:
# test_df.info()

In [ ]:
# sample_df = pd.read_csv("../input/landmark-recognition-2021/sample_submission.csv")
# sample_df

In [ ]:
# sample_df.info()